<a href="https://colab.research.google.com/github/sampogna/matriz_confusao_cancer_mama/blob/main/cancer_de_mama_TRAB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando útlima versão da lib *xlrd*

In [ ]:
%pip install --upgrade xlrd

Importando bibliotecas que serão utilizadas

In [ ]:

import pandas 
import numpy
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from statistics import mean

from sklearn.svm import SVC


A base foi convertida de *.xls* para *.csv* para facilitar a leitura com a biblioteca.
Inicializando o *dataset* que será utilizado

In [ ]:
file_name = 'BreastTissue.xls'
dataset_bt = pandas.read_excel(file_name, 'Data')

Dataset gerado

In [ ]:
dataset_bt

,Case #,Class,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
0,1,car,524.794072,0.187448,0.032114,228.800228,6843.598481,29.910803,60.204880,220.737212,556.828334
1,2,car,330.000000,0.226893,0.265290,121.154201,3163.239472,26.109202,69.717361,99.084964,400.225776
2,3,car,551.879287,0.232478,0.063530,264.804935,11888.391827,44.894903,77.793297,253.785300,656.769449
3,4,car,380.000000,0.240855,0.286234,137.640111,5402.171180,39.248524,88.758446,105.198568,493.701814
4,5,car,362.831266,0.200713,0.244346,124.912559,3290.462446,26.342127,69.389389,103.866552,424.796503
...,...,...,...,...,...,...,...,...,...,...,...
101,102,adi,2000.000000,0.106989,0.105418,520.222649,40087.920984,77.059161,204.090347,478.517223,2088.648870
102,103,adi,2600.000000,0.200538,0.208043,1063.441427,174480.476218,164.071543,418.687286,977.552367,2664.583623
103,104,adi,1600.000000,0.071908,-0.066323,436.943603,12655.342135,28.963331,103.732704,432.129749,1475.371534
104,105,adi,2300.000000,0.045029,0.136834,185.446044,5086.292497,27.427344,178.691742,49.593290,2480.592151


In [ ]:
dataset_bt.describe().round(2)

,Case #,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
count,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00
mean,53.50,784.25,0.12,0.11,190.57,7335.16,23.47,75.38,166.71,810.64
std,30.74,753.95,0.07,0.10,190.80,18580.31,23.35,81.35,181.31,763.02
min,1.00,103.00,0.01,-0.07,19.65,70.43,1.60,7.97,-9.26,124.98
25%,27.25,250.00,0.07,0.04,53.85,409.65,8.18,26.89,41.78,270.22
50%,53.50,384.94,0.11,0.09,120.78,2219.58,16.13,44.22,97.83,454.11
75%,79.75,1487.99,0.17,0.17,255.33,7615.20,30.95,83.67,232.99,1301.56
max,106.00,2800.00,0.36,0.47,1063.44,174480.48,164.07,436.10,977.55,2896.58


In [ ]:
dataset_bt = dataset_bt.to_numpy()
X = dataset_bt[:, 2:]
y = dataset_bt[:, 1]


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

Definindo função para realizar soma de duas matrizes de mesmo tamanho

In [ ]:
def soma_matrizes(m1, m2):
    matriz_soma = []
    
    quant_linhas = len(m1) 
    quant_colunas = len(m1[0])
    for i in range(quant_linhas):
        
        matriz_soma.append([])
        for j in range(quant_colunas):
            
            soma = m1[i][j] + m2[i][j]
            matriz_soma[i].append(soma)
    return matriz_soma

Definindo função que printa a matriz 6x6 de confusão

In [ ]:
def print_matrix_confusion(matrix):
  print("\n")
  for i in range(5):
    for j in range(5):
      print(matrix[i][j],end=' ')
    print("\n")

Definindo função que imprime estatísticas dos algoritmos no *output*

In [ ]:
def print_estatisticas(dic, ehKNN):
  if (ehKNN == True):
    print("\n===================== ..::"," KNN - k = ", dic["k"], " ::.. =========================\n")
  else:
    print("\n=========================== ..::"," SVM "," ::.. ===================================\n")
  
  print("Média de acurácia para o caso de teste: ", dic["acuracia_media"])
  print("Precisão: ", dic["precisao"])
  print("Revocação: ", dic["revocacao"])
  print("Matrizes de confusão somadas: \n")
  print_matrix_confusion(dic["matriz_confusao"])
  print("\n====================================================================================\n")

Definindo função que utiliza as bibliotecas dos algoritmos **KNN** e **SVM**


In [ ]:
def retorna_estatisticas(k = 0, ehKNN = True):

  # Retorna um dic com as estatisticas dos algoritmos
  
  acuracias = []
  precisoes = []
  revocacoes = []
  matriz_confusao = [[0,0,0,0,0,0],
                     [0,0,0,0,0,0],
                     [0,0,0,0,0,0],
                     [0,0,0,0,0,0],
                     [0,0,0,0,0,0],
                     [0,0,0,0,0,0]]

  for treino, teste in kf.split(X, y):
    X_treino = X[treino]
    X_teste = X[teste]
    y_treino = y[treino]
    y_teste = y[teste]
    scaler = preprocessing.StandardScaler().fit(X_treino)

    X_treino = scaler.transform(X_treino)
    X_teste = scaler.transform(X_teste)

    if (ehKNN == True):
      knn = KNeighborsClassifier(n_neighbors=k)
      knn.fit(X_treino, y_treino)
      y_pred = knn.predict(X_teste)
    else:
      svm = SVC(kernel="linear")
      svm.fit(X_treino, y_treino)
      y_pred = svm.predict(X_teste)
    
    acuracias.append(accuracy_score(y_teste, y_pred))
    precisao = precision_score(y_teste, y_pred, average='macro', zero_division=1)
    revocacao = precision_score(y_teste, y_pred, average='macro', zero_division=1)

    precisoes.append(precisao)
    revocacoes.append(revocacao)

    matriz_confusao = soma_matrizes(matriz_confusao, confusion_matrix(y_teste, y_pred))



  return {
      "acuracia_media": mean(acuracias),
      "matriz_confusao": matriz_confusao,
      "lst_acuracias": acuracias,
      "k": k,
      "precisao": mean(precisoes),
      "revocacao": mean(revocacoes),
  }



In [ ]:
k3 = retorna_estatisticas(3)

k5 = retorna_estatisticas(5)

k7 = retorna_estatisticas(7)

k9 = retorna_estatisticas(9)

svm = retorna_estatisticas(ehKNN = False)

print_estatisticas(k3, True)
print_estatisticas(k5, True)
print_estatisticas(k7, True)
print_estatisticas(k9, True)
print_estatisticas(svm, False)


===================== ..::  KNN - k =  3  ::.. =========================

Média de acurácia para o caso de teste:  0.6593073593073593
Precisão:  0.6888492063492063
Revocação:  0.6888492063492063
Matrizes de confusão somadas: 



19 0 3 0 0 

0 18 0 0 1 

2 0 10 1 0 

0 2 0 9 2 

0 1 0 3 10 




===================== ..::  KNN - k =  5  ::.. =========================

Média de acurácia para o caso de teste:  0.6593073593073593
Precisão:  0.6967195767195767
Revocação:  0.6967195767195767
Matrizes de confusão somadas: 



20 0 2 0 0 

0 17 0 1 1 

2 0 8 2 0 

0 1 0 10 3 

0 1 0 3 10 




===================== ..::  KNN - k =  7  ::.. =========================

Média de acurácia para o caso de teste:  0.6216450216450217
Precisão:  0.6194444444444445
Revocação:  0.6194444444444445
Matrizes de confusão somadas: 



19 0 3 0 0 

0 19 0 0 1 

1 0 8 2 0 

0 1 0 6 3 

0 1 0 2 10 




===================== ..::  KNN - k =  9  ::.. =========================

Média de acurácia para o caso de teste